In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/moremoreurgent/Final_Training_More_Urgent - Copy.csv
/kaggle/input/nlpword2vecembeddingspretrained/glove.6B.100d.txt
/kaggle/input/nlpword2vecembeddingspretrained/glove.6B.200d.txt
/kaggle/input/nlpword2vecembeddingspretrained/GoogleNews-vectors-negative300.bin
/kaggle/input/nlpword2vecembeddingspretrained/glove.6B.300d.txt
/kaggle/input/nlpword2vecembeddingspretrained/glove.6B.50d.txt


In [2]:
!pip install wordninja

     |████████████████████████████████| 541 kB 7.4 MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541552 sha256=b0022551a2daf95d92703f2ce0da12ca4026923c39ce91ffe9e6f62864e06c7d
  Stored in directory: /root/.cache/pip/wheels/8a/ff/82/e3dfc11e488be5de981117790251d3dd0c51544c8237b14499
Successfully built wordninja


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Convolution1D,GlobalMaxPool1D,Attention,Dropout,Dense,Embedding,Bidirectional,GRU,GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
from nltk.tokenize import word_tokenize
import re
import wordninja
from textblob import TextBlob
from tqdm import tqdm
tqdm.pandas()
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Bidirectional,GRU,Convolution1D,Dense,GlobalMaxPooling1D,Dropout
from sklearn.preprocessing import LabelEncoder

In [4]:
df = pd.read_csv('/kaggle/input/moremoreurgent/Final_Training_More_Urgent - Copy.csv')
df.head()

,Product,Consumer complaint narrative,Sentiment Score,Urgent,Clean Text,Target
0,"Credit reporting, credit repair services, or o...",reason- I disputed inaccurate unverified items...,22,1,reason- disputed inaccurate unverified item re...,Urgent
1,Debt collection,In 2016 I received notice of balance owing to ...,14,1,2016 received notice balance owing capital one...,Urgent
2,Mortgage,Hi my is name XXXX XXXX. I have some concerns ...,13,1,hi name concern servicer select portfolio serv...,NotUrgent
3,Mortgage,Specialized Loan Servicing ( SLS ) acquired my...,13,1,specialized loan servicing sl acquired mortgag...,Urgent
4,Mortgage,XXXX XXXX Chief Executive Officer and Presiden...,12,1,chief executive officer president select portf...,NotUrgent


In [5]:
def remove3ConsecutiveDuplicates(string): 
    val = "" 
    i = 0
    while (i < len(string)): 
        if (i < len(string) - 2 and
            string[i] * 3 == string[i:i + 3]): 
            i += 3
        else: 
            val += string[i] 
            i += 1
              
    if (len(val) == len(string)): 
        return val 
    else: 
        return remove3ConsecutiveDuplicates(val)
    
#stop_words = list(set(stopwords.words('english')))+list(punctuation)+['unknown','xx']
stop_words = ['unknown','xx']
lem = WordNetLemmatizer()

def removal(text):
    text = re.sub('XXXX',' UNKNOWN ',text)
    text = re.sub('XX/XX/','',text)
    text = re.sub('UNKNOWN   UNKNOWN','UNKNOWN',text)
    text = re.sub('\n',' ',text)
    text = re.sub('  ',' ',text)
    return text

def cleaning(text):
    text = removal(text)
    #text = text.lower()
    words = word_tokenize(text)
    words = [w for w in words if w not in stop_words]
    #words = [w for w in words if len(w)>2]
    words = [lem.lemmatize(w,'v') for w in words]
    return ' '.join(words)

def total_clean(text):
    text = text.lower()
    text = re.sub('[^A-Za-z0-9]',' ',text)
    text = cleaning(text)
    text = wordninja.split(text)
    text = ' '.join(text)
    text = remove3ConsecutiveDuplicates(text)
    #blob = TextBlob(text)
    #text = blob.correct()
    text = re.sub(r'\b\w{1,1}\b', '', text)
    #text = word_tokenize(str(text))
    #text = ' '.join([w for w in text if len(w)>1])
    #text = re.sub('sap','asap',str(text))
    return text

In [6]:
df['Total Clean'] = df['Clean Text'].progress_apply(total_clean)

100%|██████████| 5305/5305 [00:56<00:00, 94.26it/s] 


In [7]:
le = LabelEncoder()
df['Target']  = le.fit_transform(df['Target'])
df.head()

,Product,Consumer complaint narrative,Sentiment Score,Urgent,Clean Text,Target,Total Clean
0,"Credit reporting, credit repair services, or o...",reason- I disputed inaccurate unverified items...,22,1,reason- disputed inaccurate unverified item re...,1,reason dispute inaccurate unverified item repo...
1,Debt collection,In 2016 I received notice of balance owing to ...,14,1,2016 received notice balance owing capital one...,1,2016 receive notice balance owe capital one ca...
2,Mortgage,Hi my is name XXXX XXXX. I have some concerns ...,13,1,hi name concern servicer select portfolio serv...,0,hi name concern service select portfolio serv...
3,Mortgage,Specialized Loan Servicing ( SLS ) acquired my...,13,1,specialized loan servicing sl acquired mortgag...,1,specialize loan service sl acquire mortgage in...
4,Mortgage,XXXX XXXX Chief Executive Officer and Presiden...,12,1,chief executive officer president select portf...,0,chief executive officer president select portf...


In [8]:
df = df.sample(frac=1).reset_index(drop=True)
df['Target'].value_counts()

0    4326
1     979
Name: Target, dtype: int64

In [9]:
tokenizer = Tokenizer()
x = df['Total Clean']
y = df['Target']

tokenizer.fit_on_texts(x)
seq = tokenizer.texts_to_sequences(x)
pad_seq = pad_sequences(seq,maxlen = 500,padding='post',truncating='pre')

In [10]:
vocab_size = len(tokenizer.word_index)+1
vocab_size

9193

In [11]:
word2vec = KeyedVectors.load_word2vec_format('/kaggle/input/nlpword2vecembeddingspretrained/GoogleNews-vectors-negative300.bin', \
        binary=True)
print('Found %s word vectors of word2vec' % len(word2vec.vocab))

Found 3000000 word vectors of word2vec


In [12]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)

In [13]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [14]:
model = Sequential()
model.add(Embedding(vocab_size,300,input_length=500,weights = [embedding_matrix],trainable = False))
model.add(Bidirectional(GRU(32,return_sequences=True)))
#model.add(Convolution1D(32,2,activation='relu'))
#model.add(Convolution1D(64,3,activation = 'relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(32,activation = 'relu'))
#model.add(Dropout(0.2))
model.add(Dense(1,activation = 'sigmoid'))

model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(pad_seq,y,batch_size = 32,epochs = 10,validation_split = 0.10,callbacks=callbacks_list)

Train on 4774 samples, validate on 531 samples
Epoch 1/10
4768/4774 [============================>.] - ETA: 0s - loss: 0.4914 - accuracy: 0.7976
Epoch 00001: val_loss improved from inf to 0.40800, saving model to weights-improvement-01-0.84.h5
4774/4774 [==============================] - 63s 13ms/sample - loss: 0.4911 - accuracy: 0.7979 - val_loss: 0.4080 - val_accuracy: 0.8437
Epoch 2/10
4768/4774 [============================>.] - ETA: 0s - loss: 0.4082 - accuracy: 0.8173
Epoch 00002: val_loss improved from 0.40800 to 0.34497, saving model to weights-improvement-02-0.87.h5
4774/4774 [==============================] - 58s 12ms/sample - loss: 0.4084 - accuracy: 0.8171 - val_loss: 0.3450 - val_accuracy: 0.8663
Epoch 3/10
4768/4774 [============================>.] - ETA: 0s - loss: 0.3129 - accuracy: 0.8645
Epoch 00003: val_loss improved from 0.34497 to 0.32412, saving model to weights-improvement-03-0.86.h5
4774/4774 [==============================] - 62s 13ms/sample - loss: 0.3127 - ac

In [15]:
t = "I have an unresolved asap mortgage issue with banker. Please take action to resolve the same."

In [16]:
b = tokenizer.texts_to_sequences([t])
b_pad = pad_sequences(b,maxlen = 500,padding='post',truncating='pre')
results = model.predict(b_pad)
results

array([[0.99062777]], dtype=float32)

In [17]:
import pickle

# saving
with open('tokenizer_word2vec_more_more_urgent.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)